# Advanced Risk Management and Monitoring

This notebook covers:
- Real-time risk monitoring systems
- Dynamic hedging strategies
- Regime detection and adaptation
- Risk attribution analysis
- Early warning systems

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from datetime import datetime, timedelta

# Risk management libraries
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Custom modules
from src.data.loader import DataLoader
from src.risk.manager import RiskManager
from src.utils.logger import setup_logger

warnings.filterwarnings('ignore')
logger = setup_logger(__name__)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Risk management libraries loaded successfully")

## Load Data and Portfolio Results

In [ ]:
# Load market data
loader = DataLoader()
data = loader.load_data(['TSLA', 'BND', 'SPY'], '2015-07-01', '2025-07-31')
prices = data['prices']
returns = data['returns']

# Load portfolio recommendation
try:
    with open('../data/portfolio_recommendation.pkl', 'rb') as f:
        portfolio_rec = pickle.load(f)
    recommended_weights = portfolio_rec['weights']
    print("✓ Portfolio recommendation loaded")
except FileNotFoundError:
    print("✗ Using default weights")
    recommended_weights = {'TSLA': 0.4, 'BND': 0.3, 'SPY': 0.3}

# Calculate portfolio returns
weights_series = pd.Series(recommended_weights)
portfolio_returns = (returns * weights_series).sum(axis=1)

print(f"Portfolio weights: {recommended_weights}")
print(f"Analysis period: {returns.index.min()} to {returns.index.max()}")

## Regime Detection and Market State Analysis

In [ ]:
class MarketRegimeDetector:
    """
    Detect market regimes using various statistical methods
    """
    
    def __init__(self, returns_data, window=60):
        self.returns = returns_data
        self.window = window
        
    def calculate_regime_features(self):
        """
        Calculate features for regime detection
        """
        features = pd.DataFrame(index=self.returns.index)
        
        # Rolling statistics
        features['volatility'] = self.returns.rolling(self.window).std() * np.sqrt(252)
        features['mean_return'] = self.returns.rolling(self.window).mean() * 252
        features['skewness'] = self.returns.rolling(self.window).skew()
        features['kurtosis'] = self.returns.rolling(self.window).kurt()
        
        # VaR and CVaR
        features['var_95'] = self.returns.rolling(self.window).quantile(0.05)
        features['cvar_95'] = self.returns.rolling(self.window).apply(
            lambda x: x[x <= x.quantile(0.05)].mean()
        )
        
        # Trend indicators
        price_proxy = (1 + self.returns).cumprod()
        features['trend_strength'] = (
            price_proxy / price_proxy.rolling(self.window).mean() - 1
        )
        
        # Correlation with market (using SPY as proxy)
        if 'SPY' in self.returns.columns:
            spy_returns = self.returns['SPY']
            features['market_correlation'] = self.returns.rolling(self.window).corr(spy_returns)
        
        return features.dropna()
    
    def detect_regimes_kmeans(self, n_regimes=3):
        """
        Detect regimes using K-means clustering
        """
        features = self.calculate_regime_features()
        
        # Standardize features
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        # Apply K-means
        kmeans = KMeans(n_clusters=n_regimes, random_state=42, n_init=10)
        regimes = kmeans.fit_predict(features_scaled)
        
        # Create regime series
        regime_series = pd.Series(regimes, index=features.index)
        
        return regime_series, features, kmeans
    
    def analyze_regime_characteristics(self, regime_series, features):
        """
        Analyze characteristics of each regime
        """
        regime_stats = {}
        
        for regime in regime_series.unique():
            regime_mask = regime_series == regime
            regime_features = features[regime_mask]
            regime_returns = self.returns[regime_mask]
            
            stats_dict = {
                'frequency': regime_mask.sum(),
                'percentage': regime_mask.mean() * 100,
                'avg_volatility': regime_features['volatility'].mean(),
                'avg_return': regime_features['mean_return'].mean(),
                'avg_skewness': regime_features['skewness'].mean(),
                'avg_kurtosis': regime_features['kurtosis'].mean(),
                'avg_var': regime_features['var_95'].mean(),
                'worst_drawdown': self._calculate_max_drawdown(regime_returns)
            }
            
            regime_stats[f'Regime_{regime}'] = stats_dict
        
        return pd.DataFrame(regime_stats).T
    
    def _calculate_max_drawdown(self, returns_series):
        """
        Calculate maximum drawdown for a returns series
        """
        if len(returns_series) == 0:
            return 0
        
        cumulative = (1 + returns_series).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max
        return drawdown.min()

# Initialize regime detector
regime_detector = MarketRegimeDetector(portfolio_returns)

# Detect regimes
print("Detecting market regimes...")
regimes, features, kmeans_model = regime_detector.detect_regimes_kmeans(n_regimes=3)

# Analyze regime characteristics
regime_analysis = regime_detector.analyze_regime_characteristics(regimes, features)

print("\nMARKET REGIME ANALYSIS")
print("=" * 60)
print(regime_analysis.round(4))

# Label regimes based on characteristics
regime_labels = {}
for idx, row in regime_analysis.iterrows():
    if row['avg_volatility'] > regime_analysis['avg_volatility'].mean():
        if row['avg_return'] > 0:
            label = 'High Vol Bull'
        else:
            label = 'High Vol Bear'
    else:
        if row['avg_return'] > 0:
            label = 'Low Vol Bull'
        else:
            label = 'Low Vol Bear'
    
    regime_num = int(idx.split('_')[1])
    regime_labels[regime_num] = label

print(f"\nRegime Labels: {regime_labels}")

In [ ]:
# Visualize regime detection results
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Market Regime Detection Analysis', fontsize=16, fontweight='bold')

# Portfolio returns with regime coloring
ax1 = axes[0]
cumulative_returns = (1 + portfolio_returns).cumprod()

for regime in regimes.unique():
    regime_mask = regimes == regime
    regime_dates = regimes[regime_mask].index
    regime_values = cumulative_returns[regime_dates]
    
    ax1.scatter(regime_dates, regime_values, 
               label=f'{regime_labels.get(regime, f"Regime {regime}")}',
               alpha=0.7, s=10)

ax1.set_title('Portfolio Cumulative Returns by Market Regime')
ax1.set_ylabel('Cumulative Return')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rolling volatility with regime coloring
ax2 = axes[1]
rolling_vol = portfolio_returns.rolling(30).std() * np.sqrt(252)

for regime in regimes.unique():
    regime_mask = regimes == regime
    regime_dates = regimes[regime_mask].index
    regime_vol = rolling_vol[regime_dates]
    
    ax2.scatter(regime_dates, regime_vol,
               label=f'{regime_labels.get(regime, f"Regime {regime}")}',
               alpha=0.7, s=10)

ax2.set_title('Rolling Volatility by Market Regime')
ax2.set_ylabel('Annualized Volatility')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Regime transitions over time
ax3 = axes[2]
ax3.plot(regimes.index, regimes, linewidth=2, alpha=0.8)
ax3.set_title('Market Regime Transitions Over Time')
ax3.set_ylabel('Regime')
ax3.set_yticks(list(regime_labels.keys()))
ax3.set_yticklabels([regime_labels[k] for k in sorted(regime_labels.keys())])
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Regime transition matrix
transition_matrix = pd.crosstab(regimes.shift(1), regimes, normalize='index')
transition_matrix.index = [regime_labels.get(i, f'Regime {i}') for i in transition_matrix.index]
transition_matrix.columns = [regime_labels.get(i, f'Regime {i}') for i in transition_matrix.columns]

plt.figure(figsize=(10, 8))
sns.heatmap(transition_matrix, annot=True, cmap='Blues', fmt='.3f')
plt.title('Regime Transition Probability Matrix', fontsize=14, fontweight='bold')
plt.ylabel('From Regime')
plt.xlabel('To Regime')
plt.tight_layout()
plt.show()

print("\nREGIME TRANSITION PROBABILITIES")
print("=" * 50)
print(transition_matrix.round(3))

## Dynamic Risk Monitoring System

In [ ]:
class DynamicRiskMonitor:
    """
    Real-time risk monitoring and alert system
    """
    
    def __init__(self, portfolio_returns, weights, confidence_levels=[0.95, 0.99]):
        self.returns = portfolio_returns
        self.weights = weights
        self.confidence_levels = confidence_levels
        self.risk_metrics = {}
        
    def calculate_dynamic_var(self, window=60, method='historical'):
        """
        Calculate dynamic Value at Risk
        """
        var_results = {}
        
        for conf_level in self.confidence_levels:
            alpha = 1 - conf_level
            
            if method == 'historical':
                var_series = self.returns.rolling(window).quantile(alpha)
            elif method == 'parametric':
                rolling_mean = self.returns.rolling(window).mean()
                rolling_std = self.returns.rolling(window).std()
                z_score = stats.norm.ppf(alpha)
                var_series = rolling_mean + z_score * rolling_std
            elif method == 'ewma':
                # Exponentially weighted moving average
                ewm_mean = self.returns.ewm(span=window).mean()
                ewm_std = self.returns.ewm(span=window).std()
                z_score = stats.norm.ppf(alpha)
                var_series = ewm_mean + z_score * ewm_std
            
            var_results[f'VaR_{int(conf_level*100)}'] = var_series
        
        return pd.DataFrame(var_results)
    
    def calculate_expected_shortfall(self, window=60):
        """
        Calculate Expected Shortfall (Conditional VaR)
        """
        es_results = {}
        
        for conf_level in self.confidence_levels:
            alpha = 1 - conf_level
            
            def rolling_es(x):
                var_threshold = x.quantile(alpha)
                return x[x <= var_threshold].mean()
            
            es_series = self.returns.rolling(window).apply(rolling_es)
            es_results[f'ES_{int(conf_level*100)}'] = es_series
        
        return pd.DataFrame(es_results)
    
    def calculate_component_var(self, asset_returns, window=60):
        """
        Calculate component VaR for risk attribution
        """
        weights_series = pd.Series(self.weights)
        component_vars = {}
        
        for date in asset_returns.index[-window:]:
            # Get window of returns ending at this date
            window_returns = asset_returns.loc[:date].tail(window)
            
            if len(window_returns) < window:
                continue
            
            # Calculate covariance matrix
            cov_matrix = window_returns.cov()
            
            # Portfolio variance
            portfolio_var = np.dot(weights_series.T, np.dot(cov_matrix, weights_series))
            portfolio_vol = np.sqrt(portfolio_var)
            
            # Marginal VaR
            marginal_var = np.dot(cov_matrix, weights_series) / portfolio_vol
            
            # Component VaR
            component_var = weights_series * marginal_var
            
            component_vars[date] = component_var
        
        return pd.DataFrame(component_vars).T
    
    def detect_risk_anomalies(self, var_df, threshold_multiplier=2):
        """
        Detect risk anomalies and generate alerts
        """
        alerts = []
        
        for col in var_df.columns:
            var_series = var_df[col].dropna()
            
            # Calculate rolling statistics
            rolling_mean = var_series.rolling(30).mean()
            rolling_std = var_series.rolling(30).std()
            
            # Detect anomalies
            upper_threshold = rolling_mean + threshold_multiplier * rolling_std
            lower_threshold = rolling_mean - threshold_multiplier * rolling_std
            
            anomalies = var_series[(var_series > upper_threshold) | (var_series < lower_threshold)]
            
            for date, value in anomalies.items():
                alert_type = 'HIGH_RISK' if value > upper_threshold[date] else 'LOW_RISK'
                alerts.append({
                    'date': date,
                    'metric': col,
                    'value': value,
                    'threshold': upper_threshold[date] if alert_type == 'HIGH_RISK' else lower_threshold[date],
                    'alert_type': alert_type,
                    'severity': 'HIGH' if abs(value - rolling_mean[date]) > 3 * rolling_std[date] else 'MEDIUM'
                })
        
        return pd.DataFrame(alerts)

# Initialize risk monitor
risk_monitor = DynamicRiskMonitor(portfolio_returns, recommended_weights)

# Calculate dynamic risk metrics
print("Calculating dynamic risk metrics...")
var_metrics = risk_monitor.calculate_dynamic_var(window=60, method='ewma')
es_metrics = risk_monitor.calculate_expected_shortfall(window=60)
component_var = risk_monitor.calculate_component_var(returns, window=60)

# Detect risk anomalies
risk_alerts = risk_monitor.detect_risk_anomalies(var_metrics)

print(f"\nRISK MONITORING SUMMARY")
print(f"=" * 50)
print(f"Current VaR (95%): {var_metrics['VaR_95'].iloc[-1]:.4f}")
print(f"Current VaR (99%): {var_metrics['VaR_99'].iloc[-1]:.4f}")
print(f"Current ES (95%): {es_metrics['ES_95'].iloc[-1]:.4f}")
print(f"Current ES (99%): {es_metrics['ES_99'].iloc[-1]:.4f}")
print(f"Total risk alerts generated: {len(risk_alerts)}")

if len(risk_alerts) > 0:
    print(f"\nRecent HIGH severity alerts:")
    high_severity = risk_alerts[risk_alerts['severity'] == 'HIGH'].tail(5)
    for _, alert in high_severity.iterrows():
        print(f"  {alert['date'].strftime('%Y-%m-%d')}: {alert['metric']} = {alert['value']:.4f} ({alert['alert_type']})")

In [ ]:
# Visualize dynamic risk metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Dynamic Risk Monitoring Dashboard', fontsize=16, fontweight='bold')

# VaR evolution
ax1 = axes[0, 0]
ax1.plot(var_metrics.index, var_metrics['VaR_95'], label='VaR 95%', linewidth=2)
ax1.plot(var_metrics.index, var_metrics['VaR_99'], label='VaR 99%', linewidth=2)
ax1.fill_between(var_metrics.index, var_metrics['VaR_95'], var_metrics['VaR_99'], alpha=0.3)
ax1.set_title('Dynamic Value at Risk')
ax1.set_ylabel('VaR')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Expected Shortfall
ax2 = axes[0, 1]
ax2.plot(es_metrics.index, es_metrics['ES_95'], label='ES 95%', linewidth=2, color='red')
ax2.plot(es_metrics.index, es_metrics['ES_99'], label='ES 99%', linewidth=2, color='darkred')
ax2.set_title('Expected Shortfall (Conditional VaR)')
ax2.set_ylabel('Expected Shortfall')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Component VaR (most recent)
ax3 = axes[1, 0]
if not component_var.empty:
    latest_component_var = component_var.iloc[-1]
    colors = ['red', 'blue', 'green']
    bars = ax3.bar(latest_component_var.index, latest_component_var.values, color=colors, alpha=0.7)
    ax3.set_title('Component VaR (Risk Attribution)')
    ax3.set_ylabel('Component VaR')
    ax3.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, latest_component_var.values):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{value:.4f}', ha='center', va='bottom')

ax3.grid(True, alpha=0.3)

# Risk alerts timeline
ax4 = axes[1, 1]
if len(risk_alerts) > 0:
    # Count alerts by date
    alert_counts = risk_alerts.groupby(risk_alerts['date'].dt.date).size()
    ax4.bar(alert_counts.index, alert_counts.values, alpha=0.7, color='orange')
    ax4.set_title('Risk Alerts Timeline')
    ax4.set_ylabel('Number of Alerts')
    ax4.tick_params(axis='x', rotation=45)
else:
    ax4.text(0.5, 0.5, 'No Risk Alerts', ha='center', va='center', transform=ax4.transAxes)
    ax4.set_title('Risk Alerts Timeline')

ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risk attribution analysis
if not component_var.empty:
    print(f"\nRISK ATTRIBUTION ANALYSIS")
    print(f"=" * 50)
    
    # Calculate average component VaR
    avg_component_var = component_var.mean()
    total_component_var = avg_component_var.sum()
    
    print(f"Average Component VaR:")
    for asset, comp_var in avg_component_var.items():
        percentage = (comp_var / total_component_var) * 100
        weight = recommended_weights.get(asset, 0) * 100
        print(f"  {asset}: {comp_var:.6f} ({percentage:.1f}% of total risk, {weight:.1f}% weight)")
    
    # Risk concentration analysis
    risk_concentration = (avg_component_var / total_component_var).max()
    print(f"\nRisk Concentration (max single asset): {risk_concentration*100:.1f}%")
    
    if risk_concentration > 0.5:
        print("⚠️  WARNING: High risk concentration detected!")
    elif risk_concentration > 0.4:
        print("⚠️  CAUTION: Moderate risk concentration")
    else:
        print("✅ Risk is well diversified")

## Stress Testing and Scenario Analysis

In [ ]:
class AdvancedStressTester:
    """
    Advanced stress testing with multiple scenarios
    """
    
    def __init__(self, returns_data, weights):
        self.returns = returns_data
        self.weights = pd.Series(weights)
        self.portfolio_returns = (returns_data * self.weights).sum(axis=1)
        
    def historical_stress_test(self, stress_periods):
        """
        Test portfolio performance during historical stress periods
        """
        stress_results = {}
        
        for period_name, (start_date, end_date) in stress_periods.items():
            period_returns = self.portfolio_returns[
                (self.portfolio_returns.index >= start_date) & 
                (self.portfolio_returns.index <= end_date)
            ]
            
            if len(period_returns) > 0:
                cumulative_return = (1 + period_returns).prod() - 1
                volatility = period_returns.std() * np.sqrt(252)
                max_drawdown = self._calculate_max_drawdown(period_returns)
                worst_day = period_returns.min()
                
                stress_results[period_name] = {
                    'cumulative_return': cumulative_return,
                    'volatility': volatility,
                    'max_drawdown': max_drawdown,
                    'worst_day': worst_day,
                    'days': len(period_returns)
                }
        
        return pd.DataFrame(stress_results).T
    
    def monte_carlo_stress_test(self, scenarios, n_simulations=10000):
        """
        Monte Carlo stress testing with custom scenarios
        """
        results = {}
        
        for scenario_name, scenario_params in scenarios.items():
            # Extract scenario parameters
            asset_shocks = scenario_params.get('asset_shocks', {})
            correlation_shock = scenario_params.get('correlation_shock', 0)
            volatility_multiplier = scenario_params.get('volatility_multiplier', 1)
            
            # Base statistics
            base_returns = self.returns.mean()
            base_cov = self.returns.cov()
            
            # Apply shocks
            shocked_returns = base_returns.copy()
            for asset, shock in asset_shocks.items():
                if asset in shocked_returns.index:
                    shocked_returns[asset] += shock
            
            # Modify covariance matrix
            shocked_cov = base_cov * volatility_multiplier
            
            if correlation_shock != 0:
                # Increase correlations
                corr_matrix = base_cov.corr()
                vol_matrix = np.sqrt(np.diag(base_cov))
                
                # Shock correlations
                shocked_corr = corr_matrix + correlation_shock
                shocked_corr = np.clip(shocked_corr, -0.99, 0.99)
                np.fill_diagonal(shocked_corr, 1)
                
                # Reconstruct covariance matrix
                shocked_cov = np.outer(vol_matrix, vol_matrix) * shocked_corr
                shocked_cov = pd.DataFrame(shocked_cov, index=base_cov.index, columns=base_cov.columns)
            
            # Monte Carlo simulation
            np.random.seed(42)
            simulated_returns = np.random.multivariate_normal(
                shocked_returns.values, shocked_cov.values, n_simulations
            )
            
            # Calculate portfolio returns
            portfolio_sim_returns = np.dot(simulated_returns, self.weights.values)
            
            # Calculate statistics
            results[scenario_name] = {
                'mean_return': np.mean(portfolio_sim_returns),
                'volatility': np.std(portfolio_sim_returns),
                'var_95': np.percentile(portfolio_sim_returns, 5),
                'var_99': np.percentile(portfolio_sim_returns, 1),
                'expected_shortfall_95': np.mean(portfolio_sim_returns[portfolio_sim_returns <= np.percentile(portfolio_sim_returns, 5)]),
                'worst_case': np.min(portfolio_sim_returns),
                'probability_loss': np.mean(portfolio_sim_returns < 0) * 100
            }
        
        return pd.DataFrame(results).T
    
    def _calculate_max_drawdown(self, returns_series):
        """
        Calculate maximum drawdown
        """
        cumulative = (1 + returns_series).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max
        return drawdown.min()

# Initialize stress tester
stress_tester = AdvancedStressTester(returns, recommended_weights)

# Define historical stress periods
stress_periods = {
    'COVID-19 Crash': ('2020-02-20', '2020-04-30'),
    'December 2018 Selloff': ('2018-10-01', '2018-12-31'),
    'Early 2022 Correction': ('2022-01-01', '2022-06-30'),
    'August 2015 Flash Crash': ('2015-08-01', '2015-09-30')
}

# Run historical stress tests
print("Running historical stress tests...")
historical_stress = stress_tester.historical_stress_test(stress_periods)

print("\nHISTORICAL STRESS TEST RESULTS")
print("=" * 60)
print(historical_stress.round(4))

# Define Monte Carlo scenarios
mc_scenarios = {
    'Market Crash': {
        'asset_shocks': {'TSLA': -0.3, 'SPY': -0.2, 'BND': 0.05},
        'correlation_shock': 0.3,
        'volatility_multiplier': 2.0
    },
    'Interest Rate Shock': {
        'asset_shocks': {'BND': -0.15, 'SPY': -0.1, 'TSLA': -0.05},
        'correlation_shock': 0.2,
        'volatility_multiplier': 1.5
    },
    'Tech Bubble Burst': {
        'asset_shocks': {'TSLA': -0.5, 'SPY': -0.15, 'BND': 0.1},
        'correlation_shock': 0.1,
        'volatility_multiplier': 2.5
    },
    'Stagflation': {
        'asset_shocks': {'TSLA': -0.1, 'SPY': -0.1, 'BND': -0.2},
        'correlation_shock': 0.4,
        'volatility_multiplier': 1.8
    }
}

# Run Monte Carlo stress tests
print("\nRunning Monte Carlo stress tests...")
mc_stress = stress_tester.monte_carlo_stress_test(mc_scenarios)

print("\nMONTE CARLO STRESS TEST RESULTS")
print("=" * 60)
print(mc_stress.round(4))

In [ ]:
# Visualize stress test results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Advanced Stress Testing Results', fontsize=16, fontweight='bold')

# Historical stress test - cumulative returns
ax1 = axes[0, 0]
historical_stress['cumulative_return'].plot(kind='bar', ax=ax1, color='red', alpha=0.7)
ax1.set_title('Historical Stress Periods - Cumulative Returns')
ax1.set_ylabel('Cumulative Return')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Historical stress test - max drawdown
ax2 = axes[0, 1]
historical_stress['max_drawdown'].plot(kind='bar', ax=ax2, color='darkred', alpha=0.7)
ax2.set_title('Historical Stress Periods - Maximum Drawdown')
ax2.set_ylabel('Maximum Drawdown')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Monte Carlo stress test - VaR comparison
ax3 = axes[1, 0]
var_comparison = mc_stress[['var_95', 'var_99']]
var_comparison.plot(kind='bar', ax=ax3, alpha=0.7)
ax3.set_title('Monte Carlo Scenarios - VaR Comparison')
ax3.set_ylabel('Value at Risk')
ax3.tick_params(axis='x', rotation=45)
ax3.legend(['VaR 95%', 'VaR 99%'])
ax3.grid(True, alpha=0.3)

# Monte Carlo stress test - probability of loss
ax4 = axes[1, 1]
mc_stress['probability_loss'].plot(kind='bar', ax=ax4, color='orange', alpha=0.7)
ax4.set_title('Monte Carlo Scenarios - Probability of Loss')
ax4.set_ylabel('Probability of Loss (%)')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Stress test summary and recommendations
print(f"\nSTRESS TEST SUMMARY AND RECOMMENDATIONS")
print(f"=" * 60)

# Worst historical performance
worst_historical = historical_stress['cumulative_return'].min()
worst_period = historical_stress['cumulative_return'].idxmin()
print(f"Worst historical performance: {worst_historical:.2%} during {worst_period}")

# Worst Monte Carlo scenario
worst_mc_scenario = mc_stress['var_99'].idxmin()
worst_mc_var = mc_stress.loc[worst_mc_scenario, 'var_99']
print(f"Worst Monte Carlo scenario: {worst_mc_scenario} (VaR 99%: {worst_mc_var:.2%})")

# Risk recommendations
print(f"\nRISK MANAGEMENT RECOMMENDATIONS:")
print(f"-" * 40)

if worst_historical < -0.3:
    print("🔴 HIGH RISK: Portfolio showed severe losses during historical stress periods")
    print("   → Consider reducing allocation to high-volatility assets")
    print("   → Implement dynamic hedging strategies")
elif worst_historical < -0.15:
    print("🟡 MEDIUM RISK: Portfolio showed moderate stress during crisis periods")
    print("   → Monitor risk metrics closely during market volatility")
    print("   → Consider partial hedging during high-risk periods")
else:
    print("🟢 LOW RISK: Portfolio demonstrated resilience during historical stress periods")
    print("   → Continue current risk management approach")

# Scenario-specific recommendations
high_loss_prob_scenarios = mc_stress[mc_stress['probability_loss'] > 60]
if not high_loss_prob_scenarios.empty:
    print(f"\n⚠️  High loss probability scenarios detected:")
    for scenario in high_loss_prob_scenarios.index:
        prob = high_loss_prob_scenarios.loc[scenario, 'probability_loss']
        print(f"   → {scenario}: {prob:.1f}% probability of loss")

print(f"\nGeneral Recommendations:")
print(f"• Implement stop-loss orders at portfolio level")
print(f"• Consider tail risk hedging instruments")
print(f"• Regular stress testing (monthly)")
print(f"• Dynamic position sizing based on market regime")
print(f"• Maintain adequate cash reserves for rebalancing")

## Risk Management Summary and Action Plan

In [ ]:
# Generate comprehensive risk management summary
print("\n" + "="*80)
print("COMPREHENSIVE RISK MANAGEMENT SUMMARY")
print("="*80)

print(f"\n1. CURRENT RISK PROFILE:")
print(f"-" * 40)
current_var_95 = var_metrics['VaR_95'].iloc[-1]
current_var_99 = var_metrics['VaR_99'].iloc[-1]
current_es_95 = es_metrics['ES_95'].iloc[-1]

print(f"• Daily VaR (95%): {current_var_95:.2%}")
print(f"• Daily VaR (99%): {current_var_99:.2%}")
print(f"• Expected Shortfall (95%): {current_es_95:.2%}")
print(f"• Portfolio Volatility: {portfolio_returns.std() * np.sqrt(252):.2%}")

print(f"\n2. MARKET REGIME ANALYSIS:")
print(f"-" * 40)
current_regime = regimes.iloc[-1]
current_regime_label = regime_labels.get(current_regime, f'Regime {current_regime}')
print(f"• Current Market Regime: {current_regime_label}")

# Regime stability
recent_regimes = regimes.tail(30)  # Last 30 days
regime_changes = (recent_regimes != recent_regimes.shift(1)).sum()
print(f"• Regime Changes (last 30 days): {regime_changes}")
print(f"• Regime Stability: {'High' if regime_changes <= 2 else 'Medium' if regime_changes <= 5 else 'Low'}")

print(f"\n3. STRESS TEST INSIGHTS:")
print(f"-" * 40)
avg_historical_loss = historical_stress['cumulative_return'].mean()
worst_historical_loss = historical_stress['cumulative_return'].min()
avg_mc_var_99 = mc_stress['var_99'].mean()

print(f"• Average Historical Stress Loss: {avg_historical_loss:.2%}")
print(f"• Worst Historical Stress Loss: {worst_historical_loss:.2%}")
print(f"• Average Monte Carlo VaR (99%): {avg_mc_var_99:.2%}")

print(f"\n4. RISK CONCENTRATION:")
print(f"-" * 40)
if not component_var.empty:
    latest_component_var = component_var.iloc[-1]
    max_risk_asset = latest_component_var.abs().idxmax()
    max_risk_contribution = latest_component_var.abs().max() / latest_component_var.abs().sum()
    
    print(f"• Highest Risk Contributor: {max_risk_asset}")
    print(f"• Maximum Risk Concentration: {max_risk_contribution:.1%}")
    
    risk_level = "High" if max_risk_contribution > 0.6 else "Medium" if max_risk_contribution > 0.4 else "Low"
    print(f"• Risk Concentration Level: {risk_level}")

print(f"\n5. ALERT SYSTEM STATUS:")
print(f"-" * 40)
total_alerts = len(risk_alerts)
high_severity_alerts = len(risk_alerts[risk_alerts['severity'] == 'HIGH']) if len(risk_alerts) > 0 else 0
recent_alerts = len(risk_alerts[risk_alerts['date'] >= (datetime.now() - timedelta(days=7))]) if len(risk_alerts) > 0 else 0

print(f"• Total Risk Alerts: {total_alerts}")
print(f"• High Severity Alerts: {high_severity_alerts}")
print(f"• Recent Alerts (7 days): {recent_alerts}")

alert_status = "Critical" if high_severity_alerts > 5 else "Warning" if recent_alerts > 3 else "Normal"
print(f"• Alert System Status: {alert_status}")

print(f"\n6. RECOMMENDED ACTIONS:")
print(f"-" * 40)

# Generate specific recommendations based on analysis
recommendations = []

# VaR-based recommendations
if abs(current_var_99) > 0.05:  # 5% daily VaR
    recommendations.append("🔴 URGENT: Daily VaR exceeds 5% - consider immediate position reduction")
elif abs(current_var_99) > 0.03:  # 3% daily VaR
    recommendations.append("🟡 CAUTION: Daily VaR elevated - monitor closely and prepare for position adjustment")

# Regime-based recommendations
if regime_changes > 5:
    recommendations.append("⚠️  High regime instability - implement dynamic hedging")

# Stress test recommendations
if worst_historical_loss < -0.3:
    recommendations.append("📉 Historical stress tests show severe losses - diversify further")

# Risk concentration recommendations
if not component_var.empty and max_risk_contribution > 0.6:
    recommendations.append(f"⚖️  High risk concentration in {max_risk_asset} - rebalance portfolio")

# Alert-based recommendations
if alert_status == "Critical":
    recommendations.append("🚨 Critical alert status - immediate risk review required")
elif alert_status == "Warning":
    recommendations.append("⚠️  Warning alert status - enhanced monitoring recommended")

# Default recommendations if no specific issues
if not recommendations:
    recommendations = [
        "✅ Risk profile within acceptable parameters",
        "📊 Continue regular monitoring and monthly stress testing",
        "🔄 Maintain current rebalancing schedule"
    ]

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print(f"\n7. MONITORING SCHEDULE:")
print(f"-" * 40)
print(f"• Daily: VaR and portfolio returns monitoring")
print(f"• Weekly: Risk alert review and regime analysis")
print(f"• Monthly: Comprehensive stress testing")
print(f"• Quarterly: Risk model validation and recalibration")
print(f"• Semi-annually: Portfolio optimization review")

print(f"\n8. EMERGENCY PROCEDURES:")
print(f"-" * 40)
print(f"• VaR > 5%: Immediate position review and potential reduction")
print(f"• Multiple high-severity alerts: Emergency risk committee meeting")
print(f"• Regime change: Reassess portfolio allocation within 48 hours")
print(f"• Stress test failure: Implement contingency hedging strategies")

print(f"\n" + "="*80)
print("RISK MANAGEMENT ANALYSIS COMPLETE")
print("="*80)

# Save risk management results
risk_management_summary = {
    'current_risk_metrics': {
        'var_95': current_var_95,
        'var_99': current_var_99,
        'expected_shortfall_95': current_es_95,
        'portfolio_volatility': portfolio_returns.std() * np.sqrt(252)
    },
    'regime_analysis': {
        'current_regime': current_regime_label,
        'regime_stability': 'High' if regime_changes <= 2 else 'Medium' if regime_changes <= 5 else 'Low',
        'recent_changes': int(regime_changes)
    },
    'stress_test_summary': {
        'worst_historical_loss': worst_historical_loss,
        'average_mc_var_99': avg_mc_var_99
    },
    'alert_status': alert_status,
    'recommendations': recommendations
}

with open('../data/risk_management_summary.pkl', 'wb') as f:
    pickle.dump(risk_management_summary, f)

print("\nRisk management summary saved to '../data/risk_management_summary.pkl'")